In [1]:
import requests
from requests.auth import HTTPBasicAuth
from encryption import RSA, Encryption
import random

In [11]:
# view all account test
url = "http://127.0.0.1:8000/user/register"
test = False
if test:
    response = requests.get(url=url)
    print(response.json())

In [13]:

# register
# params: username, password, public key
e, d, n, z ,p , q = RSA.generate_key_pair(16)
username = "minh1"
password = "123"
public_key = e
signature = RSA.power_mod(Encryption.hash_string(username), d, n)
obj = {
    'username':username,
    'password':password,
    'public_key':public_key,
    'signature':signature,
    'n':n
}
response = requests.post(
    url=url,
    json=obj
    )
response.status_code


201

In [7]:
# register
# params: username, password, public key
e, d, n, z ,p , q = RSA.generate_key_pair(16)
username = input("Username: ")
password = input("Password: ")
public_key = e
signature = RSA.power_mod(Encryption.hash_string(username), d, n)
obj = {
    'username':username,
    'password':password,
    'public_key':public_key,
    'signature':signature,
    'n':n
}
response = requests.post(
    url=url,
    json=obj
    )
response.status_code


201

In [30]:
# get all images of user
url = "http://127.0.0.1:8000/user/images"
username = "minh1"
password = "123"
response = requests.get(
    url=url,
    auth=HTTPBasicAuth(username=username,password=password))
print(response.json())


[{'id': 9, 'image': '/media/user_images/10/pic1.jpg', 'user': 11, 'encrypted_session_key': 123}, {'id': 10, 'image': '/media/user_images/10/pic1.jpg', 'user': 11, 'encrypted_session_key': 123}, {'id': 11, 'image': '/media/user_images/10/pic1.jpg', 'user': 11, 'encrypted_session_key': 123}, {'id': 14, 'image': '/media/user_images/13/pic1.jpg', 'user': 11, 'encrypted_session_key': 0}, {'id': 15, 'image': '/media/user_images/13/pic1.jpg', 'user': 11, 'encrypted_session_key': 0}, {'id': 16, 'image': '/media/user_images/13/pic1.jpg', 'user': 11, 'encrypted_session_key': 7039}]


In [15]:
# upload images
# params: username, password, image
# path_to_file = input("Path to image: ")
path_to_file = "encryption/img/pic1.jpg"
filename = path_to_file.split('/')[-1]

key = random.randint(0, 256)
encrypted_session_key = RSA.power_mod(key, e, n)

Encryption.encrypt_image(path_to_file, filename,key)

files = {'image': open(filename,'rb')}
data = {'encrypted_session_key':encrypted_session_key}

response = requests.post(
    url=url,
    auth=HTTPBasicAuth(username=username,password=password),
    files=files,
    data=data
    )
print(response)

<Response [201]>


In [45]:
# download image
username = "minh1"
password = "123"

response = requests.get(
    url=url,
    auth=HTTPBasicAuth(username=username,password=password))
print(response.json())
for obj in response.json():
    if obj['id'] == "16":
        encrypted_session_key = obj['encrypted_session_key']
        break
response = requests.get(url=url+"/16", stream=True, auth=HTTPBasicAuth(username=username,password=password))
if response.ok:

    with open('downloaded.jpg', 'wb') as handle:

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)

[{'id': 3, 'image': '/media/user_images/8/pic1.jpg', 'user': 8, 'encrypted_session_key': 20647}, {'id': 4, 'image': '/media/user_images/8/pic1_SImLxmU.jpg', 'user': 8, 'encrypted_session_key': 3244}]


In [29]:
username = "minh3"
password = "123"
url = "http://127.0.0.1:8000/user/images/share"
payload = {
    "target_username":"minh1",
    "img_id":"13",
}

r = requests.get(
    url=url,
    params=payload,
    auth=HTTPBasicAuth(username=username,password=password)
)
if r.ok:
    data = r.json()
    target_pub_key = int(data['target_pub_key'])
    encrypted_session_key = int(data['encrypted_session_key'])
    request_user_n = int(data['request_user_n'])
    target_user_n = int(data['target_user_n'])
    print(target_pub_key)
    print(encrypted_session_key)
    with open(f"{username}_key.txt","r") as prv_key_file:
        prv_key = int(prv_key_file.read())
        print(prv_key)
        session_key = RSA.power_mod(encrypted_session_key, prv_key, request_user_n)
        print(session_key)
    re_encrypted_key = RSA.power_mod(session_key, target_pub_key, target_user_n)
    print(re_encrypted_key)
    payload = {
        "re_encrypted_key":re_encrypted_key,
        "img_id":"13",
        "target_username":"minh1",
    }

    r = requests.post(
    url=url,
    data=payload,
    auth=HTTPBasicAuth(username=username,password=password)
    )




19507
6306
14081
146
7039


In [ ]:
e = 7121
d = 14081
